In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import print_function
# Import các thư viện để hỗ trợ đọc, ghi, kiểm tra và Preprocess data
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

np.random.seed(1337)  # for reproducibility
# Import các thư viện để thực hiện các bước Label Encode, Scale, Feature Selection và Resampling dữ liệu 
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import f_classif, SelectKBest
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline

# Import các thư viện cần thiết để tiến hành phân tách phục vụ đào tạo tập dữ liệu
from sklearn.model_selection import train_test_split

# Import các thư viện cần thiết để tạo, save và load model 
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Activation, Lambda, Embedding
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, load_model
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, f1_score, mean_absolute_error, make_scorer

# Import các thư viện để đánh giá các tiêu chí và vẽ confusion matrix
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import balanced_accuracy_score, f1_score, precision_score, recall_score, accuracy_score, classification_report

In [ ]:
# Hàm vẽ Confusion Matrix
def plot_cm(y_true, y_pred, title, labels):
    figsize=(7,7)
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    plt.title(title)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax)

In [ ]:
# Đọc dữ liệu cần train
data = pd.read_csv("/content/drive/MyDrive/IDS_Test_Train/Datasets/CIC-18/cicids2018.csv")
# Đọc dữ liệu tự tạo data1 sẽ là dữ liệu thuần Fingerprinting và data 2 là dữ liệu thuần Bình thường
data1 = pd.read_csv("/content/drive/MyDrive/IDS_Test_Train/Datasets/CIC-IDS-2017/nmap_exploit.csv")
data2 = pd.read_csv("/content/drive/MyDrive/IDS_Test_Train/Datasets/HKUST/Normal.csv")
# Gán nhãn cho tập dữ liệu tự tạo
data1.Label = "Infilteration"
data2.Label = "Benign"
# gắn lại tạo thành một tập dữ liệu tự tạo duy nhất
test_data = pd.concat([data1, data2], axis=0)

In [ ]:
# Data Preprocess
test_data = test_data[test_data['Src IP'].str.contains('137.66.32.0|8.6.0.1')==False]

In [ ]:
# Data Preprocess
data.drop(["Protocol"], axis=1, inplace=True)
test_data.drop(["Flow ID","Timestamp", "Src Port", "Protocol"], axis=1, inplace=True)

In [ ]:
# Để thuận tiện trong quá trình đào tạo tập dữ liệu
attack = data[data['Label'] == 'Infilteration']
normal = data[data['Label'] == 'Benign'].sample(frac=0.1)
data = pd.concat([attack, normal], axis=0)

In [ ]:
# Xóa các dữ liệu có giá trị NULL, +-INF bộ data tự tạo
test_data.replace([np.inf, -np.inf], np.nan, inplace=True)
test_data.dropna(inplace=True)
test_data.drop_duplicates(inplace=True)
test_data.reset_index(inplace=True, drop=True)

In [ ]:
# Lấy flow, src và dst IP để ráp lại sau khi predict bộ test_data
flow = pd.DataFrame()
flow['Src IP'] = test_data['Src IP']
flow['Dst IP'] = test_data['Dst IP']
test_data.drop(["Src IP", "Dst IP"], axis=1, inplace=True)

In [ ]:
# Xóa các dữ liệu có giá trị NULL, +-INF
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True, drop=True)

In [ ]:
# Kiểm tra số lượng nhãn
print ("Data Label:")
print (data.Label.value_counts())
print ("Predict Data Label:")
print (test_data.Label.value_counts())

In [ ]:
data.columns = test_data.columns

In [ ]:
# Label Encode nhãn tập dữ liệu sử dụng để train
le = LabelEncoder()
data.Label = le.fit_transform(data.Label)
data.Label.value_counts()

In [ ]:
# Label Encode nhãn cho tập dữ liệu cần predict
test_data.Label = le.fit_transform(test_data.Label)
test_data.Label.value_counts()

In [ ]:
# Tách dữ liệu ra để tiến hành train/ predict
X = data.drop("Label", axis=1)
Y = data.Label.copy()
X_use = test_data.drop("Label", axis=1)
Y_use = test_data.Label.copy()

In [ ]:
# Scale data X
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_use = scaler.transform(X_use)

In [ ]:
#Feature selection 
selectKBest = SelectKBest(score_func=f_classif, k=64)
X_selected = selectKBest.fit_transform(X, Y)
X_use_selected = selectKBest.transform(X_use)

In [ ]:
# Tách dữ liệu gốc thành dữ liệu train/ test với tỉ lệ 80 % train / 20% test
X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y, test_size=0.2, random_state=101)

In [ ]:
#Resampling dataset
resampling = SMOTETomek("auto")
steps = [("SMOTETomek",resampling)]
pipeline = Pipeline(steps=steps)
X_train_resampled, Y_train_resampled = pipeline.fit_resample(X_train, Y_train)

In [ ]:
# Xây dựng dnn model
model = Sequential()
model.add(Dense(256,input_dim=X_train_resampled.shape[1],activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(128,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(64,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train_resampled, Y_train_resampled, validation_data=(X_test, Y_test),batch_size=64, epochs=100)

In [ ]:
# Predict lại bộ test cho ra kết quả
Y_pred = model.predict(X_test)

In [ ]:
# Xử lý theo hàm activation cuối là sigmoid
Y_pred = map(lambda x: 0 if x<0.5 else 1, Y_pred)
Y_pred = list(Y_pred)

In [ ]:
# Confusion matrix cho bộ test
plot_cm(Y_test, Y_pred, 'Confusion matrix for predictions on the test dataset', [0,1])
#Xuất kết quả predict bộ test 20%
report = classification_report(Y_test, Y_pred, digits=3)
print (report)

In [ ]:
# Predict lưu lượng mạng thu thập
test_pred = model.predict(X_use_selected)

In [ ]:
# Xử lý theo hàm activation cuối là sigmoid với threshold là 0.5
test_pred = map(lambda x: 0 if x<0.5 else 1, test_pred)
test_pred = list(test_pred) 

In [ ]:
# Tạo data từ mảng test_pred thành dạng Cột Predict
prediction = pd.DataFrame(test_pred, columns=['Predict']) 
# Ghép flow với cột Predict
flow = flow.reset_index(drop=True)
full_pred = pd.concat([flow, prediction], axis=1)
# Xác định dữ liệu attack
attack = full_pred[full_pred.Predict == 1] 

In [ ]:
# Phát hiện ra Src IP nào tấn công, địa chỉ tấn công chính xác là 10.0.0.1
attack["Src IP"].value_counts()

In [ ]:
# Confusion matrix cho tập dữ liệu thu thập được
plot_cm(Y_use, test_pred, 'Confusion matrix for predictions on the collected dataset', [0,1])

In [ ]:
#Xuất kết quả bộ dữ liệu thu thập được
report = classification_report(Y_use, test_pred, digits=3)
print (report)